# Problem Statement
The Star Wars API has several different kinds of output that I hope to turn into dataframes for easy manipulation in Python. Once a request is made to the API, it returns data in JSON format, which isn't ideal for data analysis and modelling approaches.

There are several JSON types that the API returns:
- Films
- Planets
- People
- Species
- Starships 
- Vehicles

We can turn each of these results into individual dataframes. 

This assignment consists of two main steps
1. Using Python to make API requests to get data
2. Converting the JSON output of the API into a data frame

I'll be using the Python requests and pandas libraries to perform these.

Let's start by importing these libraries:

In [39]:
import requests as rq
import pandas as pd

## What does the data look like?

First lets look at what the data looks like when we make a request. 

In [40]:
response = rq.get("https://swapi.co/api/people/")
print(response)

<Response [200]>


###### That's not the response we were hoping for. Let's access the json response by the jons() method which produces a dictionary of the json response. Let's see what we get





In [41]:
print(response.json())

{'count': 87, 'next': 'https://swapi.co/api/people/?page=2', 'previous': None, 'results': [{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.co/api/planets/1/', 'films': ['https://swapi.co/api/films/2/', 'https://swapi.co/api/films/6/', 'https://swapi.co/api/films/3/', 'https://swapi.co/api/films/1/', 'https://swapi.co/api/films/7/'], 'species': ['https://swapi.co/api/species/1/'], 'vehicles': ['https://swapi.co/api/vehicles/14/', 'https://swapi.co/api/vehicles/30/'], 'starships': ['https://swapi.co/api/starships/12/', 'https://swapi.co/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.co/api/people/1/'}, {'name': 'C-3PO', 'height': '167', 'mass': '75', 'hair_color': 'n/a', 'skin_color': 'gold', 'eye_color': 'yellow', 'birth_year': '112BBY', 'gender': 'n/a', 'homeworld':

## What? It breaks things into several pages!

It still has some extra stuff, such as count, and links to the next page of responses. On top of that, we can't get all the data in one API request. We have information on the count of people '87' from the ouput this request. This could be useful. 

Let's try to get data of one person at a time and see where that gets us.

In [42]:
response = rq.get("https://swapi.co/api/people/1")
responseDict = response.json()
print(responseDict)

{'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'https://swapi.co/api/planets/1/', 'films': ['https://swapi.co/api/films/2/', 'https://swapi.co/api/films/6/', 'https://swapi.co/api/films/3/', 'https://swapi.co/api/films/1/', 'https://swapi.co/api/films/7/'], 'species': ['https://swapi.co/api/species/1/'], 'vehicles': ['https://swapi.co/api/vehicles/14/', 'https://swapi.co/api/vehicles/30/'], 'starships': ['https://swapi.co/api/starships/12/', 'https://swapi.co/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'https://swapi.co/api/people/1/'}


## And the Force Awakens! 

We know the total count of people, and the API request link is easy to iterate over as the person ID can easily be passed to the api request function. Let's do that later, since we have established the logic behind it.

We need to solve another piece of the puzzle. How do we turn that piece of JSON into a dataframe? 

We turned the JSON object from the API into a dictionary, but now we need to convert it to a dataframe

Lets get the headers of the dictionary as our column names for the data frame. Let's see what those are by looking at the keys of the dictionary we got


In [43]:
PeopleHeaders = response.json().keys()
print(PeopleHeaders)

dict_keys(['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color', 'birth_year', 'gender', 'homeworld', 'films', 'species', 'vehicles', 'starships', 'created', 'edited', 'url'])


Now we have column headers. Let's start accessing each person with an API request, convert JSON into dictionary, access the value for each key and place it in a row, then we append it to a main people array. As we do this, we also access the links within each JSON to get the names of the values which only have links. Let's repeat some code from before:

In [44]:
response = rq.get("https://swapi.co/api/people/")
NumPeople = response.json()['count']

response = rq.get("https://swapi.co/api/people/1")
PeopleHeaders = response.json().keys()

## The meat and potatoes

Now we have to go to each person, go to each of the other links within their output and get the names/titles of those things. This was just a lot of trial and error to get to an acceptable solution. 

Here's the gist of the algorithm:

1. Create an empty array
2. Create a secondary empty array that will be appended to primary array
3. Access a person's API output
4. Check each key/value pair to see if there are links or lists, if its a list, create a tertiary array, and we already know those lists only have links in them.
5. access the name/title key using a secondary api request using the information from link or list of links, add to tertiary list if necessary
6. append info to secondary array
7. append info to primary array
8. convert primary array to a data frame using pandas

### Final deliverable

A dataframe with no urls to get to valuable information. No need for a relational database, as the list within each row will have all the information necessary, without having to go through another link.

There will be a couple of moving parts that process the API response to a dataframe.
Aside from the flow explained earlier, there are some extra checks:

1. To find the headers of the dataframe, check several responses to make sure they are not empty (404 errors are common)
2. In every iteration of finding an entry, check if the response is not a 404
3. Dont use the link for an API request if the key of that dictionary is 'url'

After creating an array of all the data, a dataframe can be created with pandas. We can pass the array and the headers to pandas.DataFrame() to get a proper dataframe.

Since the api request urls are very similar so we can use a similar approach to get data on species, starships, planets, and vehicles. We can create a function that can take a string and pass it to the api request to get different portions of data. 

So here's a function that can do all those things, with a couple of comments to explain the major steps.

In [45]:
def GetSWAPIDataFrame(string):
    
    # function takes string 
    import requests as rq
    import pandas as pd
    
    # test a couple of responses and get the headers of the response that isn't empty
    for check in range(1,20):
        response = rq.get("https://swapi.co/api/"+string+"/"+str(check))
        if response.status_code != 404:
            responseDict = response.json()
            Headers = response.json().keys()
            print("Headers = ", list(Headers))
            break
    
    # the response when no ID is passes gives us the count of entries, which we use to iterate over
    response = rq.get("https://swapi.co/api/"+string+"/")
    Num = response.json()['count']
    
    # define main array as an empty array
    Array = []
    
    # iterate over the number of entries, pass the iterable into the url of the request
    for ID in range(1,Num+1):
        # create an empty array for each entry
        Row = []
        response = rq.get("https://swapi.co/api/"+string+"/"+str(ID))
        responseDict = response.json()
        # check that response is not empty
        if response.status_code != 404:
            # check for each header and keep the same order as the headers extracted earlier
            for Header in Headers:
                # check if a part of the entry is a list, then it's most definitely just links
                if (type(responseDict[Header]) == list):
                    # check each link, fetch the data from each link, append this list of names to array for each entry
                    itemNames = []
                    for item in responseDict[Header]:
                        # the film table is weird. it uses 'title' instead of 'name'
                        if Header == 'films':
                            print("*")
                            itemNames.append(rq.get(item).json()['title'])
                        else:
                            print("*")
                            itemNames.append(rq.get(item).json()['name'])
                    Row.append(itemNames)
                    #print(itemNames)    
                elif (Header != 'url') and (type(responseDict[Header]) == str) and (responseDict[Header][:4] == 'http'):
                    print("*")
                    Row.append(rq.get(responseDict[Header]).json()['name'])
                else:
                    Row.append(responseDict[Header])

        if Row != []:
            #print(Row)
            print("Got All info for "+str(Row[0]))
            Array.append(Row)
    
    DF = pd.DataFrame(Array, columns = Headers)
    
    return DF

# The future use

The above function will be able to create a dataframe from the API. It can take the following strings as arguments and create different dataframes accordingly

- 'people'
- 'species'
- 'vehicles'
- 'starships'
- 'planets'
- 'films'

For example:

In [46]:
GetSWAPIDataFrame('films')

Headers =  ['title', 'episode_id', 'opening_crawl', 'director', 'producer', 'release_date', 'characters', 'planets', 'starships', 'vehicles', 'species', 'created', 'edited', 'url']
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for A New Hope
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for The Empire Strikes Back
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Return of the Jedi
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for The Phantom Menace
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Attack of the Clones
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[Luke Skywalker, C-3PO, R2-D2, Darth Vader, Le...","[Alderaan, Yavin IV, Tatooine]","[CR90 corvette, Star Destroyer, Sentinel-class...","[Sand Crawler, T-16 skyhopper, X-34 landspeede...","[Hutt, Wookiee, Droid, Human, Rodian]",2014-12-10T14:23:31.880000Z,2015-04-11T09:46:52.774897Z,https://swapi.co/api/films/1/
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[Luke Skywalker, C-3PO, R2-D2, Darth Vader, Le...","[Hoth, Dagobah, Bespin, Ord Mantell]","[Executor, Millennium Falcon, Y-wing, X-wing, ...","[TIE/LN starfighter, Snowspeeder, TIE bomber, ...","[Yoda's species, Trandoshan, Wookiee, Droid, H...",2014-12-12T11:26:24.656000Z,2017-04-19T10:57:29.544256Z,https://swapi.co/api/films/2/
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"[Luke Skywalker, C-3PO, R2-D2, Darth Vader, Le...","[Dagobah, Endor, Naboo, Coruscant, Tatooine]","[Executor, Millennium Falcon, Y-wing, X-wing, ...","[TIE/LN starfighter, TIE bomber, AT-AT, AT-ST,...","[Human, Droid, Wookiee, Hutt, Yoda's species, ...",2014-12-18T10:39:33.255000Z,2015-04-11T09:46:05.220365Z,https://swapi.co/api/films/3/
3,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"[C-3PO, R2-D2, Obi-Wan Kenobi, Anakin Skywalke...","[Naboo, Coruscant, Tatooine]","[Naboo Royal Starship, Scimitar, Republic Crui...","[Vulture Droid, Multi-Troop Transport, Armored...","[Human, Droid, Yoda's species, Neimodian, Gung...",2014-12-19T16:52:55.740000Z,2015-04-11T09:45:18.689301Z,https://swapi.co/api/films/4/
4,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"[C-3PO, R2-D2, Owen Lars, Beru Whitesun lars, ...","[Naboo, Coruscant, Kamino, Geonosis, Tatooine]","[Slave 1, Naboo fighter, J-type diplomatic bar...","[Sand Crawler, Zephyr-G swoop bike, Koro-2 Exo...","[Kaminoan, Skakoan, Droid, Togruta, Yoda's spe...",2014-12-20T10:57:57.886000Z,2015-04-11T09:45:01.623982Z,https://swapi.co/api/films/5/
5,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"[Luke Skywalker, C-3PO, R2-D2, Darth Vader, Le...","[Alderaan, Dagobah, Naboo, Coruscant, Utapau, ...","[Jedi starfighter, Trade Federation cruiser, T...","[Vulture Droid, LAAT/i, AT-TE, Neimoidian shut...","[Toong, Skakoan, Droid, Wookiee, Kaleesh, Pau'...",2014-12-20T18:49:38.403000Z,2015-04-11T09:45:44.862122Z,https://swapi.co/api/films/6/
6,The Force Awakens,7,Luke Skywalker has vanished.\r\nIn his absence...,J. J. Abrams,"Kathleen Kennedy, J. J. Abrams, Bryan Burk",2015-12-11,"[Luke Skywalker, R2-D2, Leia Organa, Chewbacca...",[Jakku],"[T-70 X-wing fighter, Millennium Falcon]",[],"[Wookiee, Droid, Human]",2015-04-17T06:51:30.504780Z,2015-12-17T14:31:47.617768Z,https://swapi.co/api/films/7/


In [47]:
GetSWAPIDataFrame('species')

Headers =  ['name', 'classification', 'designation', 'average_height', 'skin_colors', 'hair_colors', 'eye_colors', 'average_lifespan', 'homeworld', 'language', 'people', 'films', 'created', 'edited', 'url']
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Human
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Droid
*
*
*
*
*
*
*
*
Got All info for Wookiee
*
*
*
Got All info for Rodian
*
*
*
*
Got All info for Hutt
*
*
*
*
*
*
*
Got All info for Yoda's species
*
*
*
Got All info for Trandoshan
*
*
*
Got All info for Mon Calamari
*
*
*
Got All info for Ewok
*
*
*
Got All info for Sullustan
*
*
*
Got All info for Neimodian
*
*
*
*
*
*
Got All info for Gungan
*
*
*
*
Got All info for Toydarian
*
*
*
Got All info for Dug
*
*
*
*
*
*
*
Got All info for Twi'lek
*
*
*
Got All info for Aleena
*
*
*
Got All info for Vulptereen
*
*
*
Got All info for Xexto
*
*
*
*
Got All info for Toong
*
*
*
*
Got All info for Cerean
*
*
*
Got All info for Nautolan
*

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,Coruscant,Galactic Basic,"[Luke Skywalker, Darth Vader, Leia Organa, Owe...","[The Empire Strikes Back, The Force Awakens, A...",2014-12-10T13:52:11.567000Z,2015-04-17T06:59:55.850671Z,https://swapi.co/api/species/1/
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,"[C-3PO, R2-D2, R5-D4, IG-88, BB8]","[The Empire Strikes Back, The Force Awakens, A...",2014-12-10T15:16:16.259000Z,2015-04-17T06:59:43.869528Z,https://swapi.co/api/species/2/
2,Wookiee,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,Kashyyyk,Shyriiwook,"[Chewbacca, Tarfful]","[The Empire Strikes Back, The Force Awakens, R...",2014-12-10T16:44:31.486000Z,2015-01-30T21:23:03.074598Z,https://swapi.co/api/species/3/
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,Rodia,Galactic Basic,[Greedo],[A New Hope],2014-12-10T17:05:26.471000Z,2016-07-19T13:27:03.156498Z,https://swapi.co/api/species/4/
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,Nal Hutta,Huttese,[Jabba Desilijic Tiure],"[Return of the Jedi, A New Hope]",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.co/api/species/5/
5,Yoda's species,mammal,sentient,66,"green, yellow","brown, white","brown, green, yellow",900,unknown,Galactic basic,[Yoda],"[The Empire Strikes Back, Attack of the Clones...",2014-12-15T12:27:22.877000Z,2014-12-20T21:36:42.148000Z,https://swapi.co/api/species/6/
6,Trandoshan,reptile,sentient,200,"brown, green",none,"yellow, orange",unknown,Trandosha,Dosh,[Bossk],[The Empire Strikes Back],2014-12-15T13:07:47.704000Z,2014-12-20T21:36:42.151000Z,https://swapi.co/api/species/7/
7,Mon Calamari,amphibian,sentient,160,"red, blue, brown, magenta",none,yellow,unknown,Mon Cala,Mon Calamarian,[Ackbar],[Return of the Jedi],2014-12-18T11:09:52.263000Z,2014-12-20T21:36:42.153000Z,https://swapi.co/api/species/8/
8,Ewok,mammal,sentient,100,brown,"white, brown, black","orange, brown",unknown,Endor,Ewokese,[Wicket Systri Warrick],[Return of the Jedi],2014-12-18T11:22:00.285000Z,2014-12-20T21:36:42.155000Z,https://swapi.co/api/species/9/
9,Sullustan,mammal,sentient,180,pale,none,black,unknown,Sullust,Sullutese,[Nien Nunb],[Return of the Jedi],2014-12-18T11:26:20.103000Z,2014-12-20T21:36:42.157000Z,https://swapi.co/api/species/10/


In [48]:
GetSWAPIDataFrame('people')

Headers =  ['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color', 'birth_year', 'gender', 'homeworld', 'films', 'species', 'vehicles', 'starships', 'created', 'edited', 'url']
*
*
*
*
*
*
*
*
*
*
*
Got All info for Luke Skywalker
*
*
*
*
*
*
*
*
Got All info for C-3PO
*
*
*
*
*
*
*
*
*
Got All info for R2-D2
*
*
*
*
*
*
*
Got All info for Darth Vader
*
*
*
*
*
*
*
*
Got All info for Leia Organa
*
*
*
*
*
Got All info for Owen Lars
*
*
*
*
*
Got All info for Beru Whitesun lars
*
*
*
Got All info for R5-D4
*
*
*
*
Got All info for Biggs Darklighter
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Obi-Wan Kenobi
*
*
*
*
*
*
*
*
*
*
Got All info for Anakin Skywalker
*
*
*
*
Got All info for Wilhuff Tarkin
*
*
*
*
*
*
*
*
*
*
Got All info for Chewbacca
*
*
*
*
*
*
*
*
Got All info for Han Solo
*
*
*
Got All info for Greedo
*
*
*
*
*
Got All info for Jabba Desilijic Tiure
*
*
*
*
*
*
*
Got All info for Wedge Antilles
*
*
*
*
Got All info for Jek Tono Porkins
*
*
*
*
*
*
*
Got All in

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,"[The Empire Strikes Back, Revenge of the Sith,...",[Human],"[Snowspeeder, Imperial Speeder Bike]","[X-wing, Imperial shuttle]",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.co/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,Tatooine,"[The Empire Strikes Back, Attack of the Clones...",[Droid],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.co/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,Naboo,"[The Empire Strikes Back, Attack of the Clones...",[Droid],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.co/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,"[The Empire Strikes Back, Revenge of the Sith,...",[Human],[],[TIE Advanced x1],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.co/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,"[The Empire Strikes Back, Revenge of the Sith,...",[Human],[Imperial Speeder Bike],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.co/api/people/5/
5,Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,Tatooine,"[Attack of the Clones, Revenge of the Sith, A ...",[Human],[],[],2014-12-10T15:52:14.024000Z,2014-12-20T21:17:50.317000Z,https://swapi.co/api/people/6/
6,Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,Tatooine,"[Attack of the Clones, Revenge of the Sith, A ...",[Human],[],[],2014-12-10T15:53:41.121000Z,2014-12-20T21:17:50.319000Z,https://swapi.co/api/people/7/
7,R5-D4,97,32,n/a,"white, red",red,unknown,n/a,Tatooine,[A New Hope],[Droid],[],[],2014-12-10T15:57:50.959000Z,2014-12-20T21:17:50.321000Z,https://swapi.co/api/people/8/
8,Biggs Darklighter,183,84,black,light,brown,24BBY,male,Tatooine,[A New Hope],[Human],[],[X-wing],2014-12-10T15:59:50.509000Z,2014-12-20T21:17:50.323000Z,https://swapi.co/api/people/9/
9,Obi-Wan Kenobi,182,77,"auburn, white",fair,blue-gray,57BBY,male,Stewjon,"[The Empire Strikes Back, Attack of the Clones...",[Human],[Tribubble bongo],"[Jedi starfighter, Trade Federation cruiser, N...",2014-12-10T16:16:29.192000Z,2014-12-20T21:17:50.325000Z,https://swapi.co/api/people/10/


In [49]:
GetSWAPIDataFrame('planets')

Headers =  ['name', 'rotation_period', 'orbital_period', 'diameter', 'climate', 'gravity', 'terrain', 'surface_water', 'population', 'residents', 'films', 'created', 'edited', 'url']
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Tatooine
*
*
*
*
*
Got All info for Alderaan
*
Got All info for Yavin IV
*
Got All info for Hoth
*
*
*
Got All info for Dagobah
*
*
Got All info for Bespin
*
*
Got All info for Endor
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
Got All info for Naboo
*
*
*
*
*
*
*
Got All info for Coruscant
*
*
*
*
Got All info for Kamino
*
*
Got All info for Geonosis
*
*
Got All info for Utapau
*
Got All info for Mustafar
*
*
*
Got All info for Kashyyyk
*
Got All info for Polis Massa
*
Got All info for Mygeeto
*
Got All info for Felucia
*
*
Got All info for Cato Neimoidia
*
Got All info for Saleucami
*
Got All info for Stewjon
*
Got All info for Eriadu
*
*
Got All info for Corellia
*
Got All info for Rodia
*
Got All info for Nal Hutta
Got All info for Dantooine
*
Got All info for Bestine IV

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[Luke Skywalker, C-3PO, Darth Vader, Owen Lars...","[Attack of the Clones, The Phantom Menace, Rev...",2014-12-09T13:50:49.641000Z,2014-12-21T20:48:04.175778Z,https://swapi.co/api/planets/1/
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[Leia Organa, Bail Prestor Organa, Raymus Anti...","[Revenge of the Sith, A New Hope]",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.co/api/planets/2/
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[A New Hope],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.co/api/planets/3/
3,Hoth,23,549,7200,frozen,1.1 standard,"tundra, ice caves, mountain ranges",100,unknown,[],[The Empire Strikes Back],2014-12-10T11:39:13.934000Z,2014-12-20T20:58:18.423000Z,https://swapi.co/api/planets/4/
4,Dagobah,23,341,8900,murky,N/A,"swamp, jungles",8,unknown,[],"[The Empire Strikes Back, Revenge of the Sith,...",2014-12-10T11:42:22.590000Z,2014-12-20T20:58:18.425000Z,https://swapi.co/api/planets/5/
5,Bespin,12,5110,118000,temperate,"1.5 (surface), 1 standard (Cloud City)",gas giant,0,6000000,[Lobot],[The Empire Strikes Back],2014-12-10T11:43:55.240000Z,2014-12-20T20:58:18.427000Z,https://swapi.co/api/planets/6/
6,Endor,18,402,4900,temperate,0.85 standard,"forests, mountains, lakes",8,30000000,[Wicket Systri Warrick],[Return of the Jedi],2014-12-10T11:50:29.349000Z,2014-12-20T20:58:18.429000Z,https://swapi.co/api/planets/7/
7,Naboo,26,312,12120,temperate,1 standard,"grassy hills, swamps, forests, mountains",12,4500000000,"[R2-D2, Palpatine, Jar Jar Binks, Roos Tarpals...","[Attack of the Clones, The Phantom Menace, Rev...",2014-12-10T11:52:31.066000Z,2014-12-20T20:58:18.430000Z,https://swapi.co/api/planets/8/
8,Coruscant,24,368,12240,temperate,1 standard,"cityscape, mountains",unknown,1000000000000,"[Finis Valorum, Adi Gallia, Jocasta Nu]","[Attack of the Clones, The Phantom Menace, Rev...",2014-12-10T11:54:13.921000Z,2014-12-20T20:58:18.432000Z,https://swapi.co/api/planets/9/
9,Kamino,27,463,19720,temperate,1 standard,ocean,100,1000000000,"[Boba Fett, Lama Su, Taun We]",[Attack of the Clones],2014-12-10T12:45:06.577000Z,2014-12-20T20:58:18.434000Z,https://swapi.co/api/planets/10/


In [50]:
GetSWAPIDataFrame('starships')

Headers =  ['name', 'model', 'manufacturer', 'cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'consumables', 'hyperdrive_rating', 'MGLT', 'starship_class', 'pilots', 'films', 'created', 'edited', 'url']
*
*
*
Got All info for CR90 corvette
*
*
*
Got All info for Star Destroyer
*
Got All info for Sentinel-class landing craft
*
Got All info for Death Star
*
*
*
*
*
*
*
*
Got All info for Millennium Falcon
*
*
*
Got All info for Y-wing
*
*
*
*
*
*
*
Got All info for X-wing
*
*
Got All info for TIE Advanced x1
*
*
Got All info for Executor
*
*
Got All info for Rebel transport
*
*
*
Got All info for Slave 1
*
*
*
*
*
Got All info for Imperial shuttle
*
*
Got All info for EF76 Nebulon-B escort frigate
*
Got All info for Calamari Cruiser
*
*
Got All info for A-wing
*
Got All info for B-wing
*
Got All info for Republic Cruiser
*
*
*
Got All info for Droid control ship


,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000,150,950,165,600,3000000,1 year,2.0,60,corvette,[],"[Revenge of the Sith, Return of the Jedi, A Ne...",2014-12-10T14:20:33.369000Z,2014-12-22T17:35:45.408368Z,https://swapi.co/api/starships/2/
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,47060,0,36000000,2 years,2.0,60,Star Destroyer,[],"[The Empire Strikes Back, Return of the Jedi, ...",2014-12-10T15:08:19.848000Z,2014-12-22T17:35:44.410941Z,https://swapi.co/api/starships/3/
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000,38,1000,5,75,180000,1 month,1.0,70,landing craft,[],[A New Hope],2014-12-10T15:48:00.586000Z,2014-12-22T17:35:44.431407Z,https://swapi.co/api/starships/5/
3,Death Star,DS-1 Orbital Battle Station,"Imperial Department of Military Research, Sien...",1000000000000,120000,n/a,342953,843342,1000000000000,3 years,4.0,10,Deep Space Mobile Battlestation,[],[A New Hope],2014-12-10T16:36:50.509000Z,2014-12-22T17:35:44.452589Z,https://swapi.co/api/starships/9/
4,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,100000,34.37,1050,4,6,100000,2 months,0.5,75,Light freighter,"[Chewbacca, Han Solo, Lando Calrissian, Nien N...","[The Empire Strikes Back, The Force Awakens, R...",2014-12-10T16:59:45.094000Z,2014-12-22T17:35:44.464156Z,https://swapi.co/api/starships/10/
5,Y-wing,BTL Y-wing,Koensayr Manufacturing,134999,14,1000km,2,0,110,1 week,1.0,80,assault starfighter,[],"[The Empire Strikes Back, Return of the Jedi, ...",2014-12-12T11:00:39.817000Z,2014-12-22T17:35:44.479706Z,https://swapi.co/api/starships/11/
6,X-wing,T-65 X-wing,Incom Corporation,149999,12.5,1050,1,0,110,1 week,1.0,100,Starfighter,"[Luke Skywalker, Biggs Darklighter, Wedge Anti...","[The Empire Strikes Back, Return of the Jedi, ...",2014-12-12T11:19:05.340000Z,2014-12-22T17:35:44.491233Z,https://swapi.co/api/starships/12/
7,TIE Advanced x1,Twin Ion Engine Advanced x1,Sienar Fleet Systems,unknown,9.2,1200,1,0,150,5 days,1.0,105,Starfighter,[Darth Vader],[A New Hope],2014-12-12T11:21:32.991000Z,2014-12-22T17:35:44.549047Z,https://swapi.co/api/starships/13/
8,Executor,Executor-class star dreadnought,"Kuat Drive Yards, Fondor Shipyards",1143350000,19000,n/a,279144,38000,250000000,6 years,2.0,40,Star dreadnought,[],"[The Empire Strikes Back, Return of the Jedi]",2014-12-15T12:31:42.547000Z,2017-04-19T10:56:06.685592Z,https://swapi.co/api/starships/15/
9,Rebel transport,GR-75 medium transport,"Gallofree Yards, Inc.",unknown,90,650,6,90,19000000,6 months,4.0,20,Medium transport,[],"[The Empire Strikes Back, Return of the Jedi]",2014-12-15T12:34:52.264000Z,2014-12-22T17:35:44.680838Z,https://swapi.co/api/starships/17/


In [51]:
GetSWAPIDataFrame('vehicles')

Headers =  ['name', 'model', 'manufacturer', 'cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'consumables', 'vehicle_class', 'pilots', 'films', 'created', 'edited', 'url']
*
*
Got All info for Sand Crawler
*
Got All info for T-16 skyhopper
*
Got All info for X-34 landspeeder
*
*
*
Got All info for TIE/LN starfighter
*
*
*
Got All info for Snowspeeder
*
*
Got All info for TIE bomber
*
*
Got All info for AT-AT
*
*
*
Got All info for AT-ST
*
Got All info for Storm IV Twin-Pod cloud car
*
Got All info for Sail barge
*
Got All info for Bantha-II cargo skiff
*
Got All info for TIE/IN interceptor
*
*
*
Got All info for Imperial Speeder Bike
*
*
Got All info for Vulture Droid
*
Got All info for Multi-Troop Transport
*
Got All info for Armored Assault Tank
*
Got All info for Single Trooper Aerial Platform
*
Got All info for C-9979 landing craft
*
*
*
Got All info for Tribubble bongo


,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[Attack of the Clones, A New Hope]",2014-12-10T15:36:25.724000Z,2014-12-22T18:21:15.523587Z,https://swapi.co/api/vehicles/4/
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,[],[A New Hope],2014-12-10T16:01:52.434000Z,2014-12-22T18:21:15.552614Z,https://swapi.co/api/vehicles/6/
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,unknown,repulsorcraft,[],[A New Hope],2014-12-10T16:13:52.586000Z,2014-12-22T18:21:15.583700Z,https://swapi.co/api/vehicles/7/
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,unknown,6.4,1200,1,0,65,2 days,starfighter,[],"[The Empire Strikes Back, Return of the Jedi, ...",2014-12-10T16:33:52.860000Z,2014-12-22T18:21:15.606149Z,https://swapi.co/api/vehicles/8/
4,Snowspeeder,t-47 airspeeder,Incom corporation,unknown,4.5,650,2,0,10,none,airspeeder,"[Luke Skywalker, Wedge Antilles]",[The Empire Strikes Back],2014-12-15T12:22:12Z,2014-12-22T18:21:15.623033Z,https://swapi.co/api/vehicles/14/
5,TIE bomber,TIE/sa bomber,Sienar Fleet Systems,unknown,7.8,850,1,0,none,2 days,space/planetary bomber,[],"[The Empire Strikes Back, Return of the Jedi]",2014-12-15T12:33:15.838000Z,2014-12-22T18:21:15.667730Z,https://swapi.co/api/vehicles/16/
6,AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,20,60,5,40,1000,unknown,assault walker,[],"[The Empire Strikes Back, Return of the Jedi]",2014-12-15T12:38:25.937000Z,2014-12-22T18:21:15.714673Z,https://swapi.co/api/vehicles/18/
7,AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Milit...",unknown,2,90,2,0,200,none,walker,[Chewbacca],"[The Empire Strikes Back, Return of the Jedi]",2014-12-15T12:46:42.384000Z,2014-12-22T18:21:15.761584Z,https://swapi.co/api/vehicles/19/
8,Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin Motors,75000,7,1500,2,0,10,1 day,repulsorcraft,[],[The Empire Strikes Back],2014-12-15T12:58:50.530000Z,2014-12-22T18:21:15.783232Z,https://swapi.co/api/vehicles/20/
9,Sail barge,Modified Luxury Sail Barge,Ubrikkian Industries Custom Vehicle Division,285000,30,100,26,500,2000000,Live food tanks,sail barge,[],[Return of the Jedi],2014-12-18T10:44:14.217000Z,2014-12-22T18:21:15.807906Z,https://swapi.co/api/vehicles/24/


## Conclusion

A function to extract all the data from each of the different sections of the url. The function prints the entries that it has extracted data for. It returns a pandas dataframe. 